In [2]:
from langchain.vectorstores.neo4j_vector import Neo4jVector
from langchain.embeddings.openai import OpenAIEmbeddings
import os
from langchain.chains import GraphCypherQAChain
from langchain.agents import initialize_agent, AgentType, Tool
from langchain_openai import ChatOpenAI
from langchain.graphs import Neo4jGraph


In [22]:
def get_index(index, name):
    url = os.getenv("NEO4J_URL")
    usr = os.getenv("NEO4J_USERNAME")
    pwd = os.getenv("NEO4J_PASSWORD")

    existing_index = Neo4jVector.from_existing_index(
        OpenAIEmbeddings(),
        url=url,
        username=usr,
        password=pwd,
        index_name=f"{index}",
        text_node_property=f"{name}"

    )

In [3]:

def get_cypher():
    url = os.getenv("NEO4J_URL")
    usr = os.getenv("NEO4J_USERNAME")
    pwd = os.getenv("NEO4J_PASSWORD")

    graph = Neo4jGraph(
    url=url,
    username=usr,
    password=pwd)



    cypher_runnable = GraphCypherQAChain.from_llm(
        cypher_llm = ChatOpenAI(model="gpt-4"),
        qa_llm = ChatOpenAI(model="gpt-4"),
        graph=graph,
        verbose=True
    )
    return cypher_runnable

tools = [
    Tool(name="Graph Query",
         func=get_cypher().run,
         description="Useful for finding relations in a graph"
         )
]
agent = initialize_agent(
    tools=tools,
    llm=ChatOpenAI(model="gpt-4o"),
    agent_type=AgentType.OPENAI_FUNCTIONS,
    verbose=True
)
agent.run("How many papers are published by Chulalongkorn University?")

/var/folders/cc/rnt3w6p15_9bhmg17qtpnjlw0000gn/T/ipykernel_83029/171196798.py:27: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 1.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  agent = initialize_agent(
/var/folders/cc/rnt3w6p15_9bhmg17qtpnjlw0000gn/T/ipykernel_83029/171196798.py:33: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  agent.run("How many papers are published by Chulalongkorn University?")




> Entering new AgentExecutor chain...
I need to find the number of papers published by Chulalongkorn University. To do this, I will query the graph database to get the relevant information.
Action: Graph Query
Action Input: {"institution": "Chulalongkorn University", "output": "number_of_papers"}

> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (a:Author)-[:AFFILIATED_WITH]->(o:Organization {name: "Chulalongkorn University"}), (a)-[:AUTHORED]->(l:Literature)
RETURN COUNT(l) AS number_of_papers
Full Context:
[{'number_of_papers': 4948}]

> Finished chain.

Observation: Chulalongkorn University has produced 4948 papers.
Thought:I now know the final answer.
Final Answer: Chulalongkorn University has published 4948 papers.

> Finished chain.


'Chulalongkorn University has published 4948 papers.'

In [28]:
r = get_cypher()